# QM-only Calculation 

In this example, we will use QMzyme to create a truncated active site composed of residues within 4 Angstroms of the bound ligand. All C-alpha atoms are set to be frozen during geometry optimization.

Classes used in this example:

- [GenerateModel](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.GenerateModel.html)

- [QM_Method](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.CalculateModel.html#qm-treatment)

- [CA_terminal TruncationScheme](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.TruncationSchemes.html#QMzyme.TruncationSchemes.CA_terminal)

- [DistanceCutoff SelectionScheme](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.SelectionSchemes.html#QMzyme.SelectionSchemes.DistanceCutoff)


In [1]:
from QMzyme import GenerateModel
from QMzyme.CalculateModel import QM_Method
from QMzyme.data import PDB

/Users/hrk/anaconda3/envs/qmzyme/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


#### Initialize Model and Define Regions

In [2]:
model = GenerateModel(PDB)
model.set_catalytic_center(selection='resid 263')

from QMzyme.SelectionSchemes import DistanceCutoff
model.set_region(selection=DistanceCutoff, cutoff=5)
print(model.regions)

[<QMzymeRegion catalytic_center contains 37 atom(s) and 1 residue(s)>, <QMzymeRegion cutoff_5 contains 427 atom(s) and 33 residue(s)>]


#### Truncate Region

In [3]:
model.truncate_region(region=model.cutoff_5)
print(model.regions)

[<QMzymeRegion catalytic_center contains 37 atom(s) and 1 residue(s)>, <QMzymeRegion cutoff_5 contains 427 atom(s) and 33 residue(s)>, <QMzymeRegion cutoff_5_truncated contains 391 atom(s) and 33 residue(s)>]


#### Designate Coordinate Constraints
Make sure this is done with the truncated region!

In [4]:
c_alpha_atoms = model.cutoff_5_truncated.get_atoms(attribute='name', value='CA')
model.cutoff_5_truncated.set_fixed_atoms(atoms=c_alpha_atoms)

print(model.cutoff_5_truncated.get_atoms(attribute='is_fixed', value=True))

[<QMzymeAtom 236: CA of resname TYR, resid 16>, <QMzymeAtom 258: CA of resname ILE, resid 17>, <QMzymeAtom 311: CA of resname VAL, resid 20>, <QMzymeAtom 596: CA of resname ASP, resid 40>, <QMzymeAtom 617: CA of resname PRO, resid 41>, <QMzymeAtom 853: CA of resname TYR, resid 57>, <QMzymeAtom 897: CA of resname GLN, resid 59>, <QMzymeAtom 914: CA of resname GLY, resid 60>, <QMzymeAtom 922: CA of resname LEU, resid 61>, <QMzymeAtom 984: CA of resname VAL, resid 66>, <QMzymeAtom 1024: CA of resname ALA, resid 68>, <QMzymeAtom 1227: CA of resname MET, resid 84>, <QMzymeAtom 1258: CA of resname PHE, resid 86>, <QMzymeAtom 1302: CA of resname VAL, resid 88>, <QMzymeAtom 1333: CA of resname MET, resid 90>, <QMzymeAtom 1463: CA of resname LEU, resid 99>, <QMzymeAtom 1494: CA of resname VAL, resid 101>, <QMzymeAtom 1529: CA of resname ASH, resid 103>, <QMzymeAtom 1745: CA of resname MET, resid 116>, <QMzymeAtom 1779: CA of resname ALA, resid 118>, <QMzymeAtom 1810: CA of resname TRP, resid 12

#### Build the QM Method and Assign to Region

In [5]:
qm_method = QM_Method(
    basis_set='6-31G*', 
    functional='wB97X-D3', 
    qm_input='OPT FREQ', 
    program='orca'
)

qm_method.assign_to_region(region=model.cutoff_5_truncated)


Estimating total charge for QMzymeRegion cutoff_5_truncated based on protein residue naming conventions...
<QMzymeResidue resname: TYR, resid: 16, chain: A> Charge: 0
<QMzymeResidue resname: ILE, resid: 17, chain: A> Charge: 0
<QMzymeResidue resname: VAL, resid: 20, chain: A> Charge: 0
<QMzymeResidue resname: ASP, resid: 40, chain: A> Charge: -1
<QMzymeResidue resname: PRO, resid: 41, chain: A> Charge: 0
<QMzymeResidue resname: TYR, resid: 57, chain: A> Charge: 0
<QMzymeResidue resname: GLN, resid: 59, chain: A> Charge: 0
<QMzymeResidue resname: GLY, resid: 60, chain: A> Charge: 0
<QMzymeResidue resname: LEU, resid: 61, chain: A> Charge: 0
<QMzymeResidue resname: VAL, resid: 66, chain: A> Charge: 0
<QMzymeResidue resname: ALA, resid: 68, chain: A> Charge: 0
<QMzymeResidue resname: MET, resid: 84, chain: A> Charge: 0
<QMzymeResidue resname: PHE, resid: 86, chain: A> Charge: 0
<QMzymeResidue resname: VAL, resid: 88, chain: A> Charge: 0
<QMzymeResidue resname: MET, resid: 90, chain: A> C

#### Adjust Region Charge
I know that the charge of EQU is -1, so I know that the estimated total charge is actually off. This can be manually adjusted.

In [6]:
model.cutoff_5_truncated.set_charge(-2)

#### Write QM Input File

In [7]:
model.write_input()

File /Users/hrk/git/QMzyme-documentation/docs/Examples/QCALC/cutoff_5_truncated.inp created.
